In [1]:
import json
import string
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer

import numpy as np
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [2]:
num_words = 10000
def create_tokenizer(num_words):

    with open('annotations/captions_train2017.json', 'r') as f:
        datastore = json.load(f)
        
    exclude = set(string.punctuation) 
    exclude.remove("'")

    aggregated = {}
    caption_list =  []
    image_id = []
    for captions in datastore['annotations']:
        s = captions['caption']
        s = ''.join(ch for ch in s if ch not in exclude)
        s = "SS " + s + " EE"
        caption_list.append(s)
        image_id.append(captions['image_id'])
        
        if captions['image_id'] not in aggregated:
            aggregated[captions['image_id']] = [s]
        else:
            aggregated[captions['image_id']].append(s) 
        
 
    return caption_list, image_id

In [3]:
caption_list, train_image_id = create_tokenizer(10000)

In [6]:
caption_list[0:10]

['SS A bicycle replica with a clock as the front wheel EE',
 'SS A room with blue walls and a white sink and door EE',
 'SS A car that seems to be parked illegally behind a legally parked car EE',
 'SS A large passenger airplane flying through the air EE',
 'SS There is a GOL plane taking off in a partly cloudy sky EE',
 'SS Blue and white color scheme in a small bathroom EE',
 'SS This is a blue and white bathroom with a wall sink and a lifesaver on the wall EE',
 'SS A blue boat themed bathroom with a life preserver on the wall EE',
 'SS The bike has a clock as a tire EE',
 'SS two cars parked on the sidewalk on the street EE']

In [4]:
t = Tokenizer(num_words = 10000)
t.fit_on_texts(caption_list)
encoded_docs = t.texts_to_sequences(caption_list)

In [28]:
used_words =  [item for sublist in encoded_docs for item in sublist]
used_words = set(used_words)

In [20]:
vocab_size = 10000

In [11]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.array(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [32]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    if i in used_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [35]:
embedding_matrix.shape

(10000, 100)

In [ ]:
#decoder_embedding = Embedding(input_dim=10000,
#                              output_dim=100,
#                               weights=[embedding_matrix])